In [ ]:
import os
import requests
pdf_path = "Diffusion_Models.pdf"
if not os.path.exists(pdf_path):
  url = "https://arxiv.org/pdf/2208.11970"
  filename = pdf_path
  response = requests.get(url)
  if response.status_code == 200:
    with open(filename, 'wb') as f:
      f.write(response.content)
  else:
    print(f"Failed to download the file. Status code: {response.status_code}")
else:
  print("file {pdf_path} exists.")

file {pdf_path} exists.


In [2]:
import fitz
from tqdm.auto import tqdm
def text_formatter(text: str)-> str:
  cleaned_text = text.replace("\n","").strip()
  return cleaned_text
def open_and_read_pdf(pdf_path: str)-> list[dict]:
  doc = fitz.open(pdf_path)
  pages_and_texts = []
  for page_number,page in tqdm(enumerate(doc)):
    text = page.get_text()
    text = text_formatter(text)
    pages_and_texts.append({"page_number": page_number ,
                            "page_char_count": len(text),
                            "page_word_count": len(text.split(". ")),
                            "page_sentence_count_raw": len(text.split(". ")),
                            "page_token_count": len(text)/4,
                            "text":text})
  return pages_and_texts
pages_and_texts = open_and_read_pdf(pdf_path=pdf_path)
pages_and_texts[:2]

/media/data_dump/conda/miniconda3/envs/SARAL/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
23it [00:00, 145.17it/s]


[{'page_number': 0,
  'page_char_count': 2500,
  'page_word_count': 469,
  'page_sentence_count_raw': 469,
  'page_token_count': 625.0,
  'text': 'Understanding Diﬀusion Models: A Uniﬁed PerspectiveCalvin LuoGoogle Research, Brain Teamcalvinluo@google.comAugust 26, 2022ContentsIntroduction: Generative Models . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .1Background: ELBO, VAE, and Hierarchical VAE . . . . . . . . . . . . . . . . . . . . . . . .2Evidence Lower Bound . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .2Variational Autoencoders . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .4Hierarchical Variational Autoencoders . . . . . . . . . . . . . . . . . . . . . . . . . . . . .5Variational Diﬀusion Models . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .6Learning Diﬀusion Noise Parameters . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .14Three Equivalent Interpretations. . . 

In [3]:
import pandas as pd
df = pd.DataFrame(pages_and_texts)
df.head()

,page_number,page_char_count,page_word_count,page_sentence_count_raw,page_token_count,text
0,0,2500,469,469,625.00,Understanding Diﬀusion Models: A Uniﬁed Perspe...
1,1,3828,13,13,957.00,Score-based generative models are highly relat...
2,2,2955,6,6,738.75,To make the relationship with the evidence exp...
3,3,3418,13,13,854.50,Figure 1: A Variational Autoencoder graphicall...
4,4,3788,14,14,947.00,A deﬁning feature of the VAE is how the ELBO i...


In [ ]:

df.describe().round(2)

,page_number,page_char_count,page_word_count,page_sentence_count_raw,page_token_count
count,23.00,23.00,23.00,23.00,23.00
mean,11.00,2912.52,30.91,30.91,728.13
std,6.78,779.60,95.78,95.78,194.90
min,0.00,1702.00,3.00,3.00,425.50
25%,5.50,2297.00,7.00,7.00,574.25
50%,11.00,2808.00,10.00,10.00,702.00
75%,16.50,3426.50,13.00,13.00,856.62
max,22.00,4414.00,469.00,469.00,1103.50


In [5]:
# You may need to install spaCy and its English model first:
# pip install -U spacy
# python -m spacy download en_core_web_sm

import spacy
from spacy.lang.en import English


In [6]:
# Let's use spaCy to break our text into sentences since it's likely a bit more robust than just using text.split(".")

from spacy.lang.en import English # see https://spacy.io/usage for install instructions

nlp = English()

# Add a sentencizer pipeline, see https://spacy.io/api/sentencizer/
nlp.add_pipe("sentencizer")

# Create a document instance as an example
doc = nlp("This is a sentence. This another sentence.")
assert len(list(doc.sents)) == 2

# Access the sentences of the document
list(doc.sents)

[This is a sentence., This another sentence.]

In [7]:
# Assuming 'pages_and_texts' is a list of dictionaries and 'nlp' is the spaCy model from the previous cell
from tqdm.auto import tqdm

for item in tqdm(pages_and_texts):
    item["sentences"] = list(nlp(item["text"]).sents)

    # Make sure all sentences are strings
    item["sentences"] = [str(sentence) for sentence in item["sentences"]]

    # Count the sentences
    item["page_sentence_count_spacy"] = len(item["sentences"])

100%|██████████| 23/23 [00:00<00:00, 133.28it/s]


In [8]:
# To split our groups of sentences into chunks of 10 or less, let's create a function which accepts a list as input and recursively breaks into
# down into sublists of a specified size.

# Define split size to turn groups of sentences into chunks
num_sentence_chunk_size = 10

# Create a function that recursively splits a list into desired sizes
def split_list(input_list: list,
               slice_size: int) -> list[list[str]]:
    """
    Splits the input_list into sublists of size slice_size (or as close as possible).

    For example, a list of 17 sentences would be split into two lists of [[10], [7]]
    """
    return [input_list[i:i + slice_size] for i in range(0, len(input_list), slice_size)]

# Loop through pages and texts and split sentences into chunks
for item in tqdm(pages_and_texts):
    item["sentence_chunks"] = split_list(input_list=item["sentences"],
                                         slice_size=num_sentence_chunk_size)
    item["num_chunks"] = len(item["sentence_chunks"])

100%|██████████| 23/23 [00:00<00:00, 141866.16it/s]


In [9]:
import random
random.sample(pages_and_texts,k=1)

[{'page_number': 11,
  'page_char_count': 2728,
  'page_word_count': 6,
  'page_sentence_count_raw': 6,
  'page_token_count': 682.0,
  'text': 'We have therefore derived the Gaussian form of q(xt|x0). This derivation can be modiﬁed to also yield theGaussian parameterization describing q(xt−1|x0). Now, knowing the forms of both q(xt|x0) and q(xt−1|x0),we can proceed to calculate the form of q(xt−1|xt, x0) by substituting into the Bayes rule expansion:q(xt−1|xt, x0) = q(xt|xt−1, x0)q(xt−1|x0)q(xt|x0)(71)= N(xt; √αtxt−1, (1 −αt)I)N(xt−1; √¯αt−1x0, (1 −¯αt−1)I)N(xt; √¯αtx0, (1 −¯αt)I)(72)∝exp\x1a−\x14(xt −√αtxt−1)22(1 −αt)+ (xt−1 −√¯αt−1x0)22(1 −¯αt−1)−(xt −√¯αtx0)22(1 −¯αt)\x15\x1b(73)= exp\x1a−12\x14(xt −√αtxt−1)21 −αt+ (xt−1 −√¯αt−1x0)21 −¯αt−1−(xt −√¯αtx0)21 −¯αt\x15\x1b(74)= exp\x1a−12\x14(−2√αtxtxt−1 + αtx2t−1)1 −αt+ (x2t−1 −2√¯αt−1xt−1x0)1 −¯αt−1+ C(xt, x0)\x15\x1b(75)∝exp\x1a−12\x14−2√αtxtxt−11 −αt+ αtx2t−11 −αt+x2t−11 −¯αt−1−2√¯αt−1xt−1x01 −¯αt−1\x15\x1b(76)= exp\x1a−12\x14(αt1 −α

In [10]:
import re

def contains_math(text: str) -> bool:
    math_patterns = [
        # LaTeX / functional math
        r"\\[a-zA-Z]+",                 # \frac, \log, \int, etc.
        r"\b(log|exp|sqrt|sigmoid|mean|var)\b",
        r"q[_\w\d]*\([^)]+\)",          # q(z|x), q_phi(z|x)
        r"p[_\w\d]*\([^)]+\)",          # p(x|z), p_theta(x|z)
        r"\[[^\]]+\]",                 # [log p(x)], etc.
        r"\{[^}]+\}",                  # {...} LaTeX blocks

        # Greek letters (names or Unicode)
        r"\b(alpha|beta|gamma|theta|phi|mu|sigma|lambda|eta)\b",
        r"[α-ωΑ-Ω]",                   # Unicode Greek letters

        # Variables with subscripts/superscripts
        r"\b[a-zA-Z]+\d*\_\{?[a-zA-Z0-9]+\}?",   # x_t, z_{l}, etc.
        r"\^{\(?[a-zA-Z0-9]+\)?}",     # superscripts like x^{l}

        # Unicode math symbols
        r"[∑∏∂∇∥≈→⇒⇔≠≤≥∞√±×÷∈∉∩∪∅⊂⊆⊄⊇⊃]",
        r"\bDKL\b|\bKL\b",            # KL divergence

        # Math operators
        r"[*/+=<>^]",                 # symbols often in equations
        r"\bSNR\b",                   # signal-to-noise ratio
        r"\d+\s*[*/+=<>-]\s*\d+",     # basic math ops (like 3 * 4)

        # Equations or inline formulas
        r"[a-zA-Z0-9]+\s*=\s*[^\.]+", # equations like x = something
    ]
    
    return any(re.search(pattern, text) for pattern in math_patterns)


In [14]:


def clean_malformed_unicode(text: str) -> str:
    """Remove weird Unicode characters from extracted math"""
    text = re.sub(r"[\u200b-\u200f\u202a-\u202e\u2060-\u206f\uf8e0-\uf8ff]", "", text)
    return text

def format_math_tokens(text: str) -> str:
    """Fix variable formatting like x1 → x_1, x1:T → x_{1:T}"""
    text = re.sub(r"([a-zA-Z])([0-9])", r"\1_\2", text)  # xt → x_1
    text = re.sub(r"([a-zA-Z])_([a-zA-Z])", r"\1_{\2}", text)  # x_t → x_{t}
    text = re.sub(r"([a-zA-Z])([0-9]+):([A-Z0-9]+)", r"\1_{\2:\3}", text)  # x1:T → x_{1:T}
    return text

def normalize_math_keywords(text: str) -> str:
    """Standardize math-related terms"""
    text = text.replace("ELBO", "Evidence Lower Bound (ELBO)")
    text = re.sub(r"\bDKL\b", "KL Divergence", text)
    return text

def clean_math_chunk(text: str) -> str:
    """Apply all cleaning steps to one chunk"""
    text = clean_malformed_unicode(text)
    text = format_math_tokens(text)
    text = normalize_math_keywords(text)
    return text


In [15]:
pages_and_chunks = []

for item in tqdm(pages_and_texts):
    for sentence_chunk in item["sentence_chunks"]:
        chunk_dict = {}
        chunk_dict["page_number"] = item["page_number"]

        # Join the sentences into a single string
        joined_sentence_chunk = "".join(sentence_chunk).replace("  ", " ").strip()
        joined_sentence_chunk = re.sub(r'\.([A-Z])', r'. \1', joined_sentence_chunk)
        joined_sentence_chunk = clean_math_chunk(joined_sentence_chunk)  


        # Add core fields
        chunk_dict["sentence_chunk"] = joined_sentence_chunk
        chunk_dict["chunk_char_count"] = len(joined_sentence_chunk)
        chunk_dict["chunk_word_count"] = len(joined_sentence_chunk.split(" "))
        chunk_dict["chunk_token_count"] = len(joined_sentence_chunk) / 4  # token estimate

        # Add math-aware flag
        chunk_dict["contains_math"] = contains_math(joined_sentence_chunk)

        pages_and_chunks.append(chunk_dict)


100%|██████████| 23/23 [00:00<00:00, 1113.95it/s]


In [16]:
len(pages_and_chunks)


41

In [17]:
random.sample(pages_and_chunks,k=1)

[{'page_number': 21,
  'sentence_chunk': 'We have shown that when we generalize to inﬁnite latent hierarchies, even if the encoder is trivialand the latent dimension is ﬁxed and Markovian transitions are assumed, we are still able to learn powerfulmodels of data. This suggests that further performance gains can be achieved in the case of general, deepHVAEs, where complex encoders and semantically meaningful latent spaces can be potentially learned. Acknowledgments:I would like to acknowledge Josh Dillon, Yang Song, Durk Kingma, Ben Poole,Jonathan Ho, Yiding Jiang, Ting Chen, Jeremy Cohen, and Chen Sun for reviewing drafts of this workand providing many helpful edits and comments. Thanks so much!22',
  'chunk_char_count': 664,
  'chunk_word_count': 101,
  'chunk_token_count': 166.0,
  'contains_math': False}]

In [18]:
import pandas as pd
df = pd.DataFrame(pages_and_chunks)
df.describe().round(2)

,page_number,chunk_char_count,chunk_word_count,chunk_token_count
count,41.00,41.00,41.00,41.00
mean,12.05,1672.56,244.37,418.14
std,7.72,800.43,115.90,200.11
min,0.00,192.00,26.00,48.00
25%,5.00,997.00,119.00,249.25
50%,13.00,1781.00,275.00,445.25
75%,19.00,2304.00,331.00,576.00
max,22.00,3246.00,465.00,811.50


In [19]:
# Assuming 'df' is a pandas DataFrame created from the 'pages_and_chunks' list

# Show random chunks with under 30 tokens in length
min_token_length = 150

for row in df[df["chunk_token_count"] <= min_token_length].sample(5).iterrows():
    print(f'Chunk token count: {row[1]["chunk_token_count"]} | Text: {row[1]["sentence_chunk"]}')

Chunk token count: 124.0 | Text: By combining ourparameterization of SNR in Equation 111 with our deﬁnition of SNR in Equation 109, we can also explicitlyderive elegant forms for the value of ¯αt as well as for the value of 1 −¯αt:¯αt_1 −¯αt= exp(−ωη(t))(112)∴¯αt = sigmoid(−ωη(t))(113)∴1 −¯αt = sigmoid(ωη(t))(114)These terms are necessary for a variety of computations; for example, during optimization, they are used tocreate arbitrarily noisy xt from input x_0 using the reparameterization trick, as derived in Equation 69.14
Chunk token count: 135.25 | Text: Sliced score matching: A scalable approach to densityand score estimation. In Uncertainty in Artiﬁcial Intelligence, pages 574–584. PMLR, 2020.[17] Pascal Vincent. A connection between score matching and denoising autoencoders. Neural computation, 23(7):1661–1674, 2011.[18] Jonathan Ho, Chitwan Saharia, William Chan, David J Fleet, Mohammad Norouzi, and Tim Salimans. Cascadeddiﬀusion models for high ﬁdelity image generation. J. Mach

In [20]:
import openai
import os
from uuid import uuid4

# Set API key
openai.api_key = "sk-"

# Load your chunks
sentences = df["sentence_chunk"].tolist()

# Define model
embedding_model = "text-embedding-3-small"

# Embed all sentence chunks
response = openai.embeddings.create(
    model=embedding_model,
    input=sentences
)

# Parse results
embeddings = [r.embedding for r in response.data]
ids = [str(uuid4()) for _ in range(len(sentences))]
metadatas = df[["page_number", "contains_math"]].to_dict(orient="records")




In [21]:
import openai
from tqdm import tqdm

# Define the text chunks
text_chunks = df["sentence_chunk"].tolist()

# Split into smaller batches if needed
def batch_list(input_list, batch_size=128):
    for i in range(0, len(input_list), batch_size):
        yield input_list[i:i + batch_size]

# Run in batches to avoid API limits
all_embeddings = []

for batch in tqdm(batch_list(text_chunks, batch_size=64)):
    response = openai.embeddings.create(
        model="text-embedding-3-small",
        input=batch
    )
    batch_embeddings = [r.embedding for r in response.data]
    all_embeddings.extend(batch_embeddings)

# all_embeddings is your final embedding list


1it [00:02,  2.50s/it]


In [22]:
print(len(text_chunks), len(all_embeddings))


41 41


In [23]:
import pandas as pd
import csv  

# Create DataFrame
text_chunks_and_embeddings_df = pd.DataFrame(pages_and_chunks)

# Attach embeddings
text_chunks_and_embeddings_df["embedding"] = all_embeddings

# Save safely to CSV
embeddings_df_save_path = "text_chunks_and_embeddings_df.csv"
text_chunks_and_embeddings_df.to_csv(
    embeddings_df_save_path,
    index=False,
    quoting=csv.QUOTE_ALL,  
    quotechar='"',
    escapechar='\\'
)


In [24]:
# Import saved file and view
text_chunks_and_embedding_df_load = pd.read_csv(embeddings_df_save_path)
text_chunks_and_embedding_df_load.head()

,page_number,sentence_chunk,chunk_char_count,chunk_word_count,chunk_token_count,contains_math,embedding
0,0,Understanding Diﬀusion Models: A Uniﬁed Perspe...,1129,369,282.25,False,"[0.041610248386859894, 0.011848771944642067, 0..."
1,0,.20Classiﬁer Guidance . . . . . . . . . . . . ...,1190,255,297.50,True,"[-0.010904774069786072, -0.0019216948421671987..."
2,0,Another similar approach is energy-basedmodeli...,192,26,48.00,True,"[-0.0441746823489666, 0.0025687962770462036, -..."
3,1,Score-based generative models are highly relat...,2168,313,542.00,False,"[-0.012235757894814014, 0.012445785105228424, ..."
4,1,This is because try-ing to learn a representat...,1781,267,445.25,True,"[-0.0008705609361641109, 0.01104856375604868, ..."


In [25]:
import ast
import numpy as np

# Convert 'embedding' column from string to list of floats
text_chunks_and_embedding_df_load["embedding"] = text_chunks_and_embedding_df_load["embedding"].apply(ast.literal_eval)

# Create NumPy array of embeddings
embedding_array = np.array(text_chunks_and_embedding_df_load["embedding"].to_list()).astype("float32")
embedding_array.shape


(41, 1536)

In [26]:

import numpy as np
import openai
from time import perf_counter as timer

# ---- 1. Load data ----
import pandas as pd
text_chunks_and_embedding_df_load = pd.read_csv("text_chunks_and_embeddings_df.csv")

# Convert stringified embeddings back to list of floats
import ast
text_chunks_and_embedding_df_load["embedding"] = text_chunks_and_embedding_df_load["embedding"].apply(ast.literal_eval)
embedding_array = np.array(text_chunks_and_embedding_df_load["embedding"].tolist()).astype("float32")

# ---- 2. Define and embed your query ----
query = "Explain the ELBO objective in Variational Diffusion Models"

print(f"Query: {query}")

response = openai.embeddings.create(
    model="text-embedding-3-small",  # must match document embedding model
    input=[query]
)
query_embedding = np.array(response.data[0].embedding).astype("float32")

# ---- 3. Normalize embeddings for cosine similarity ----
def normalize(vectors):
    norms = np.linalg.norm(vectors, axis=1, keepdims=True)
    return vectors / norms

embedding_array_normalized = normalize(embedding_array)
query_embedding_normalized = normalize(query_embedding.reshape(1, -1))

# ---- 4. Compute cosine similarity ----
start_time = timer()
cosine_scores = np.dot(embedding_array_normalized, query_embedding_normalized.T).flatten()
end_time = timer()
print(f"Time taken for similarity computation: {end_time - start_time:.5f} seconds.")

# ---- 5. Retrieve top-k most similar chunks ----
top_k = 5
top_indices = np.argsort(cosine_scores)[-top_k:][::-1]

print("\nTop retrieved chunks:\n")
for idx in top_indices:
    score = cosine_scores[idx]
    chunk = text_chunks_and_embedding_df_load.iloc[idx]["sentence_chunk"]
    print(f"--- Score: {score:.4f} ---\n{chunk}\n")



Query: Explain the ELBO objective in Variational Diffusion Models
Time taken for similarity computation: 0.00322 seconds.

Top retrieved chunks:

--- Score: 0.5705 ---
First, we derive Variational DiﬀusionModels as a special case of a Markovian Hierarchical Variational Autoencoder, where three key assumptionsenable tractable computation and scalable optimization of the Evidence Lower Bound (ELBO). We then prove that optimizing a VDMboils down to learning a neural network to predict one of three potential objectives: the original source imagefrom any arbitrary noisiﬁcation of it, the original source noise from any arbitrarily noisiﬁed image, or thescore function of a noisiﬁed image at any arbitrary noise level. Then, we dive deeper into what it means tolearn the score function, and connect it explicitly with the perspective of Score-based Generative Modeling. Lastly, we cover how to learn a conditional distribution using diﬀusion models. In summary, diﬀusion models have shown incredible

In [27]:
#!pip install chromadb --quiet

import chromadb
from tqdm.auto import tqdm
import numpy as np
import pandas as pd

# Load your saved DataFrame
df = pd.read_csv("text_chunks_and_embeddings_df.csv", converters={"embedding": eval})

# Initialize persistent ChromaDB client
client = chromadb.Client(chromadb.config.Settings(
    persist_directory="./saral_chroma_store"  # directory where Chroma saves
))

# Create or get a collection
collection = client.get_or_create_collection(
    name="saral_diffusion_chunks",
    metadata={"source": "Diffusion Models PDF"}
)

# Prepare data for ingestion
texts = df["sentence_chunk"].tolist()
metadatas = df[["page_number", "contains_math"]].to_dict(orient="records")
ids = [f"chunk_{i}" for i in range(len(df))]
embeddings = np.array(df["embedding"].tolist())

# Add all embeddings to the collection
collection.add(
    ids=ids,
    documents=texts,
    embeddings=embeddings,
    metadatas=metadatas
)

print(f"✅ Successfully stored {len(texts)} chunks in ChromaDB.")


✅ Successfully stored 41 chunks in ChromaDB.


In [28]:
query = "Explain the ELBO objective in Variational Diffusion Models"

# Embed the query (must match your embedding model)
query_emb = openai.embeddings.create(
    model="text-embedding-3-small",
    input=[query]
).data[0].embedding

# Retrieve top-5 relevant chunks
results = collection.query(
    query_embeddings=[query_emb],
    n_results=5
)

# Display results
for i, (doc, meta) in enumerate(zip(results["documents"][0], results["metadatas"][0])):
    print(f"\n🔹 Result {i+1} — Page {meta['page_number']}, Math: {meta['contains_math']}")
    print(doc[:400], "...")



🔹 Result 1 — Page 21, Math: False
First, we derive Variational DiﬀusionModels as a special case of a Markovian Hierarchical Variational Autoencoder, where three key assumptionsenable tractable computation and scalable optimization of the Evidence Lower Bound (ELBO). We then prove that optimizing a VDMboils down to learning a neural network to predict one of three potential objectives: the original source imagefrom any arbitrary no ...

🔹 Result 2 — Page 9, Math: True
= Eq(x_1:T |x_0)log p(xT )pθ(x_0|x_1)q(x_1|x_0)+ logTYt=2pθ(xt−1|xt)q(xt−1|xt,x_0)q(xt|x_0)(((((q(xt−1|x_0)(53)= Eq(x_1:T |x_0)"log p(xT )pθ(x_0|x_1)q(x_1|x_0)+ log q(x_1|x_0)q(xT |x_0) + logTYt=2pθ(xt−1|xt)q(xt−1|xt, x_0)#(54)= Eq(x_1:T |x_0)"log p(xT )pθ(x_0|x_1)q(xT |x_0)+TXt=2logpθ(xt−1|xt)q(xt−1|xt, x_0)#(55)= Eq(x_1:T |x_0) [log pθ(x_0|x_1)] + Eq(x_1:T |x_0)logp(xT )q(xT |x_0)+ ...

🔹 Result 3 — Page 3, Math: True
Figure 1: A Variational Autoencoder graphically represented. Here, encoder q(z|x) deﬁnes 

In [29]:
def retrieve_relevant_chunks(query: str, top_k: int = 5):
    """
    Retrieves top-k chunks relevant to the query.
    Automatically prioritizes math-heavy chunks if query looks mathematical.
    """
    import re

    # Detect if query is math-related
    math_query_terms = ["derive", "derivation", "prove", "formula", "equation", "show that"]
    is_math_query = any(re.search(rf"\b{t}\b", query.lower()) for t in math_query_terms)

    # Generate query embedding
    query_emb = openai.embeddings.create(
        model="text-embedding-3-small",
        input=[query]
    ).data[0].embedding

    # Apply math filter dynamically (only if needed)
    where_clause = {"contains_math": True} if is_math_query else None

    results = collection.query(
        query_embeddings=[query_emb],
        n_results=top_k,
        where=where_clause
    )

    chunks = []
    for doc, meta in zip(results["documents"][0], results["metadatas"][0]):
        chunks.append({
            "text": doc,
            "page_number": meta.get("page_number", "?"),
            "contains_math": meta.get("contains_math", False)
        })
    return chunks


In [30]:
import pandas as pd

# (Optional) still load df if you need metadata for inspection later
df = pd.read_csv("text_chunks_and_embeddings_df.csv", converters={"embedding": eval})

query = "Explain the ELBO objective in Variational Diffusion Models"

results = retrieve_relevant_chunks(query=query, top_k=5)

# Display the retrieved chunks
for i, res in enumerate(results, start=1):
    print(f"\n🔹 Result {i}: (Page {res['page_number']}, Math: {res['contains_math']})")
    print(res['text'][:500], "...")



🔹 Result 1: (Page 21, Math: False)
First, we derive Variational DiﬀusionModels as a special case of a Markovian Hierarchical Variational Autoencoder, where three key assumptionsenable tractable computation and scalable optimization of the Evidence Lower Bound (ELBO). We then prove that optimizing a VDMboils down to learning a neural network to predict one of three potential objectives: the original source imagefrom any arbitrary noisiﬁcation of it, the original source noise from any arbitrarily noisiﬁed image, or thescore functio ...

🔹 Result 2: (Page 9, Math: True)
= Eq(x_1:T |x_0)log p(xT )pθ(x_0|x_1)q(x_1|x_0)+ logTYt=2pθ(xt−1|xt)q(xt−1|xt,x_0)q(xt|x_0)(((((q(xt−1|x_0)(53)= Eq(x_1:T |x_0)"log p(xT )pθ(x_0|x_1)q(x_1|x_0)+ log q(x_1|x_0)q(xT |x_0) + logTYt=2pθ(xt−1|xt)q(xt−1|xt, x_0)#(54)= Eq(x_1:T |x_0)"log p(xT )pθ(x_0|x_1)q(xT |x_0)+TXt=2logpθ(xt−1|xt)q(xt−1|xt, x_0)#(55)= Eq(x_1:T |x_0) [log pθ(x_0|x_1)] + Eq(x_1:T |x_0)logp(xT )q(xT |x_0)+TXt=2Eq(x_1:T |x_0)logpθ

In [31]:
results = collection.query(
    query_embeddings=[query_emb],
    n_results=top_k,
    include=["documents", "metadatas", "distances"]
)
for doc, meta, dist in zip(results["documents"][0], results["metadatas"][0], results["distances"][0]):
    print(f" Page {meta['page_number']} | Math: {meta['contains_math']} | Distance: {dist:.4f}")
    print(doc[:400], "...\n")


 Page 21 | Math: False | Distance: 0.8590
First, we derive Variational DiﬀusionModels as a special case of a Markovian Hierarchical Variational Autoencoder, where three key assumptionsenable tractable computation and scalable optimization of the Evidence Lower Bound (ELBO). We then prove that optimizing a VDMboils down to learning a neural network to predict one of three potential objectives: the original source imagefrom any arbitrary no ...

 Page 9 | Math: True | Distance: 0.9107
= Eq(x_1:T |x_0)log p(xT )pθ(x_0|x_1)q(x_1|x_0)+ logTYt=2pθ(xt−1|xt)q(xt−1|xt,x_0)q(xt|x_0)(((((q(xt−1|x_0)(53)= Eq(x_1:T |x_0)"log p(xT )pθ(x_0|x_1)q(x_1|x_0)+ log q(x_1|x_0)q(xT |x_0) + logTYt=2pθ(xt−1|xt)q(xt−1|xt, x_0)#(54)= Eq(x_1:T |x_0)"log p(xT )pθ(x_0|x_1)q(xT |x_0)+TXt=2logpθ(xt−1|xt)q(xt−1|xt, x_0)#(55)= Eq(x_1:T |x_0) [log pθ(x_0|x_1)] + Eq(x_1:T |x_0)logp(xT )q(xT |x_0)+ ...

 Page 3 | Math: True | Distance: 0.9398
Figure 1: A Variational Autoencoder graphically represented. Here,

In [32]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import BitsAndBytesConfig
from transformers.utils import is_flash_attn_2_available
use_quantization_config = True  # Use 4-bit quantization if needed (you can change it)
# Quantization config
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16
)

# Flash Attention setup (optional, performance boost)
if is_flash_attn_2_available() and torch.cuda.get_device_capability(0)[0] >= 8:
    attn_implementation = "flash_attention_2"
else:
    attn_implementation = "sdpa"

print(f"[INFO] Using attention implementation: {attn_implementation}")



[INFO] Using attention implementation: sdpa


In [33]:


model_id = "google/gemma-7b-it"
print(f"[INFO] Using model_id: {model_id}")

tokenizer = AutoTokenizer.from_pretrained(model_id)

llm_model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
    quantization_config=quantization_config if use_quantization_config else None,
    low_cpu_mem_usage=True,
    attn_implementation=attn_implementation,
)

if not use_quantization_config:
    llm_model.to("cuda")


[INFO] Using model_id: google/gemma-7b-it


`torch_dtype` is deprecated! Use `dtype` instead!
Loading checkpoint shards: 100%|██████████| 4/4 [00:18<00:00,  4.73s/it]


In [34]:
# SARAL-style GPT-generated queries
gpt4_queries = [
    "Make a 7-slide talk for graduate students focusing on methods; include 3 speaker notes per slide and preserve key equations.",
    "Create a 90-second script in plain English summarizing the key contributions of this paper.",
    "Write a press release explaining the core findings of this paper to a general audience.",
    "Generate a 5-minute technical script to accompany a slide presentation on the paper's mathematical formulation.",
    "Make a tweet thread (10 tweets) summarizing the main points of the paper with inline LaTeX for equations.",
    "Create 5 bullet points per slide for a 10-slide deck aimed at machine learning practitioners.",
    "Summarize the paper’s introduction and related work in a style suitable for policymakers unfamiliar with AI.",
    "Generate speaker notes that include exact mathematical notation for each equation-heavy section.",
    "Create a 30-second elevator pitch version of this paper for a research pitch competition.",
    "Write a detailed walkthrough of the model section of the paper with inline LaTeX and citation to source chunks."
]
# Manually created SARAL iteration-style queries
manual_queries = [
    "Make slide 3 more visual and reduce jargon.",
    "Simplify the explanation of ELBO for a non-technical audience.",
    "Turn slide 5’s script into a short 3-bullet executive summary.",
    "Add analogies to slide 2 that help explain diffusion models.",
    "Reformat speaker notes to include numbered references to the paper’s equations.",
    "Shorten the full script to fit in a 90-second narration window.",
    "Turn the bullet points from slide 4 into LinkedIn post format.",
    "Generate a layman’s explanation of the model with no equations.",
    "Add 3 citations to slide 1's script using page numbers from the paper.",
    "Explain the limitations of the method section in a debate-style tone."

]
formula_queries = [
    "Give the final ELBO formula used in this paper, with LaTeX formatting and page reference.",
    "Summarize the key equation for the KL Divergence in this paper, with brief explanation.",
    "List all major mathematical equations used in the model section with page numbers.",
    "Extract the loss function from the method section with proper LaTeX formatting.",
    "Present the final form of the diffusion process equation as described in the paper.",
    "Derive the ELBO objective step-by-step as shown in the paper.",
    "Show a complete derivation of the KL divergence term used in optimization.",
    "Explain and derive how the reparameterization trick is applied in this model.",
    "Walk through the derivation of the variational lower bound from first principles.",
    "Derive the final expression used to estimate the transition probability in VDMs.",
    "Derive the ELBO objective function step-by-step.",
    "Prove the KL divergence term in the VAE loss.",
    "Show that ELBO is a lower bound on the log evidence.",
    "Start from Bayes rule and derive the ELBO formula for a VAE.",
    "Show full derivation of ELBO in HVAE models."

]

query_list = gpt4_queries + manual_queries + formula_queries



In [35]:
import random
query = random.choice(query_list)
print(f"Selected Query:\n{query}")


Selected Query:
Shorten the full script to fit in a 90-second narration window.


In [52]:
def prompt_formatter(query: str, context_items: list[dict], audience: str = "general") -> str:
    """
    Builds an adaptive, paper-agnostic prompt.
    Supports derivation, script, and explanation modes.
    """

    # --- Handle empty context ---
    if not context_items:
        context_items = [{"page_number": "?", "sentence_chunk": "No relevant context retrieved."}]

    context = "\n\n".join(
        f"[Page {item.get('page_number', '?')}] {item['sentence_chunk']}"
        for item in context_items
    )

    # --- Detect query intent ---
    derivation_terms = ["derive", "proof", "prove", "show that", "formula", "equation"]
    script_terms = ["script", "presentation", "talk", "slides", "narration"]
    is_derivation = any(term in query.lower() for term in derivation_terms)
    is_script = any(term in query.lower() for term in script_terms)

    # --- Base intro (prompt header) ---
    base_prompt = """
You are SARAL, a research communication assistant that generates adaptive slide decks,
scripts, and summaries from academic papers for {audience} audiences.

You will use only the context provided below to answer the query.
If the paper does NOT contain enough information to answer, explicitly say:
"Not enough information in the paper to derive or explain this."

---

### Context (from paper):
{context}

---
"""

    # --- Task-specific logic ---
    if is_derivation:
        task_instructions = """
### Derivation Task:
- Identify any equations or relationships relevant to the user query.
- If equations are available, provide a **step-by-step derivation** in LaTeX.
- Each step should be mathematically valid and based only on the paper.
- Label steps clearly: Step 1, Step 2, etc.
- If derivation context is missing, clearly say so.

Output Format:
[Slide 1]
Title: Derivation Overview
Bullets:
- Purpose of derivation
- Key starting equation
Script:
Explain the motivation.

[Slide 2]
Title: Step-by-Step Derivation
Bullets:
- Step 1 — ...
- Step 2 — ...
Script:
Show equations and logic.

[Slide 3]
Title: Final Result
Bullets:
- Derived equation
- Interpretation
Script:
Summarize result and relevance.
"""
    elif is_script:
        task_instructions = """
### Technical Script Task:
- Create 5–7 slides describing the paper’s mathematical or conceptual framework.
- Each slide must include Title, Bullets, Speaker Notes, and a Script section.
- Focus on clarity and flow.
- Use LaTeX for math if present.
- Keep explanations at an appropriate level for {audience} audiences.
"""
    else:
        task_instructions = """
### Explanation Task:
- Identify relevant parts of the context that explain the query topic.
- Provide 1 concise slide with title, bullets, and a short explanatory script.
- Keep language accessible for {audience}.
"""

    # --- Footer ---
    footer = """
---

### Query:
{query}

---

Ensure:
- All math uses LaTeX ($...$ or $$...$$).
- Use only context provided from the paper.
- Do NOT invent results not in the text.
- If uncertain, say "Not enough information in the paper."
- Begin your answer directly with [Slide 1].
"""

    # --- 🔹 AUGMENTATION happens here ---
    # Inject context, query, and audience into the template
    augmented_prompt = (base_prompt + task_instructions + footer).format(
        context=context, query=query, audience=audience
    )

    # --- Apply tokenizer chat template (Gemma-compatible) ---
    dialogue = [{"role": "user", "content": augmented_prompt}]
    prompt = tokenizer.apply_chat_template(
        conversation=dialogue,
        tokenize=False,
        add_generation_prompt=True
    )

    return prompt


In [53]:
import random

# Pick a random query
query = random.choice(query_list)
print(f"Query: {query}")

# Retrieve relevant chunks directly from ChromaDB
results = retrieve_relevant_chunks(query=query, top_k=5)

# Build context items (each already has page_number and text)
context_items = []
for res in results:
    context_items.append({
        "page_number": res.get("page_number", "?"),
        "sentence_chunk": res["text"]
    })

# Format the prompt using SARAL's math-aware, audience-adaptive template
prompt = prompt_formatter(query=query, context_items=context_items)

# Optional: Preview first 1000 characters of the prompt
print("\n--- Prompt Preview (first 1000 chars) ---\n")
print(prompt[:1000])
print("\n-----------------------------------------\n")


Query: Turn slide 5’s script into a short 3-bullet executive summary.

--- Prompt Preview (first 1000 chars) ---

<bos><start_of_turn>user
You are SARAL, a research communication assistant that generates adaptive slide decks,
scripts, and summaries from academic papers for general audiences.

You will use only the context provided below to answer the query.
If the paper does NOT contain enough information to answer, explicitly say:
"Not enough information in the paper to derive or explain this."

---

### Context (from paper):
[Page 8] Figure 4: Under our ﬁrst derivation, a VDM can be optimized by ensuring that for every intermediate xt,the posterior from the latent above it pθ(xt|xt+1) matches the Gaussian corruption of the latent before itq(xt|xt−1). In this ﬁgure, for each intermediate xt, we minimize the diﬀerence between the distributionsrepresented by the pink and green arrows. Visually, this interpretation of the Evidence Lower Bound (ELBO) is depicted in Figure 4. The cost of o

In [54]:
# --- Tokenize and move to GPU ---
device = "cuda" if torch.cuda.is_available() else "cpu"
inputs = tokenizer(prompt, return_tensors="pt").to(device)

# --- Generate output tokens ---
with torch.no_grad():
    outputs = llm_model.generate(
        **inputs,
        temperature=0.7,
        do_sample=True,
        max_new_tokens=512,
        pad_token_id=tokenizer.eos_token_id
    )

# --- Decode and clean up ---
output_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

# Ensure output starts from Slide section, not the prompt
start_idx = output_text.find("[Slide 1]")
if start_idx != -1:
    answer_only = output_text[start_idx:].strip()
else:
    answer_only = output_text.replace(prompt, "").strip()

print("\n=== SARAL Generated Output ===\n")
print(answer_only)

torch.cuda.empty_cache()



=== SARAL Generated Output ===

[Slide 1].
model
## [Slide 5] - Summary of Key Points

**Title:** Key Components of the Evidence Lower Bound (ELBO) Derivation

**Bullets:**

- The ELBO is comprised of three main terms: reconstruction, prior matching, and denoising matching.
- Each term is computed as an expectation of at most one random variable at a time, greatly reducing variance compared to the original ELBO derivation.
- The Gaussian form of the approximated denoising transition step pθ(xt−1|xt) is derived and shown to match the ground-truth denoising transition step q(xt−1|xt, x_0) as closely as possible.


In [55]:
def ask_saral_query(query: str, top_k: int = 5, show_prompt: bool = False, show_context: bool = False):
    results = retrieve_relevant_chunks(query=query, top_k=top_k)

    context_items = [
        {
            "page_number": res.get("page_number", "?"),
            "contains_math": res.get("contains_math", False),
            "sentence_chunk": res["text"],
        }
        for res in results
    ]

    prompt = prompt_formatter(query=query, context_items=context_items)

    device = "cuda" if torch.cuda.is_available() else "cpu"
    inputs = tokenizer(prompt, return_tensors="pt").to(device)

    with torch.no_grad():
        outputs = llm_model.generate(
            **inputs,
            temperature=0.7,
            do_sample=True,
            max_new_tokens=1024,
            pad_token_id=tokenizer.eos_token_id,
        )

    output_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    start_idx = output_text.find("[Slide 1]")
    if start_idx != -1:
        answer_only = output_text[start_idx:].strip()
    else:
        answer_only = output_text.replace(prompt, "").strip()

    torch.cuda.empty_cache()
    return answer_only, context_items


In [56]:
import random

query = random.choice(query_list)
print(f"\n Query: {query}\n")

answer, context_used = ask_saral_query(
    query=query,
    top_k=7,
    show_prompt=True
)

print("\n=== Final Answer ===\n")
print(answer)
print("\n=== Context Chunks Used ===\n")
for c in context_used:
    print(f"Page {c['page_number']} (Math: {c['contains_math']}): {c['sentence_chunk'][:150]}...\n")



 Query: Summarize the paper’s introduction and related work in a style suitable for policymakers unfamiliar with AI.


=== Final Answer ===

[Slide 1].
model
**[Slide 1]**

**Summary of Introduction and Related Work**

The paper introduces Variational Diffusion Models (VDMs), a novel generative model based on VAEs. VDM utilizes a Gaussian distribution to model the final latent representation and denoising transitions.

**Related Work:**

- Evidence Lower Bound (ELBO) is a key concept in VAEs that measures the evidence that a given model deviates from the true data distribution.
- Variational Autoencoders (VAEs) are generative models that approximate the true data distribution by learning a latent representation.
- Score-based Generative Models (SBGMs) estimate the score function of a model, which allows for sampling from the model's distribution.
- Cascaded Diffusion Models (CDMs) generate high-fidelity images by iteratively denoising noisy images.

**Conclusion:**

The paper introduc

In [57]:
eval_questions = [
    "What is the ELBO formula in a VAE?",
    "How does the reparameterization trick help optimization?",
    "What are the three terms of the ELBO in a Variational Diffusion Model?",
    "Why is high T problematic for ELBO variance?",
    "What is the goal of matching q(xt−1|xt, x₀) with pθ(xt−1|xt)?"
]

ground_truth_answers = [
    "ELBO = E[log p(x|z)] - KL[q(z|x) || p(z)]",
    "It rewrites stochastic sampling as a deterministic function, allowing gradients to flow through noise.",
    "1. Reconstruction term, 2. Prior matching term, 3. Consistency or denoising term.",
    "Higher T leads to more Monte Carlo estimations, increasing the variance in ELBO computation.",
    "To learn a denoising step that approximates the true posterior distribution during sampling."
]


In [58]:
import os
from openai import OpenAI

os.environ["OPENAI_API_KEY"] = "sk-
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))


In [59]:
import pandas as pd
from datasets import Dataset
from ragas import evaluate
from ragas.metrics import (
    context_precision,
    context_recall,
    answer_relevancy,
    faithfulness,
)

# Optional metrics
try:
    from ragas.metrics import context_entity_recall
except ImportError:
    context_entity_recall = None
    print("context_entity_recall not available in this version")

try:
    from ragas.metrics import noise_robustness
except ImportError:
    noise_robustness = None
    print("noise_robustness not available in this version")



noise_robustness not available in this version


In [60]:
eval_questions = [
    "Derive the ELBO objective function step-by-step.",
    "Summarize the key equation for the KL Divergence.",
    "Explain how diffusion models relate to variational inference.",
    "What is the role of the reparameterization trick in this paper?",
    "State the loss function used for optimization."
]

ground_truth_answers = [
    "The ELBO is given by E_{q(z|x)}[log p(x|z)] - KL(q(z|x) || p(z)). The derivation involves applying Jensen's inequality to log p(x).",
    "The KL Divergence measures the difference between q(z|x) and p(z) and is expressed as E_{q(z|x)}[log q(z|x) - log p(z)].",
    "Diffusion models can be interpreted as hierarchical VAEs trained via a variational lower bound.",
    "The reparameterization trick enables gradient flow through stochastic latent variables using z = μ + σϵ.",
    "The loss function minimizes the KL divergence between the forward and reverse diffusion processes."
]


In [61]:
def generate_saral_answer(query):
    """Generate SARAL answer for RAG evaluation"""
    # Retrieve relevant chunks
    results = retrieve_relevant_chunks(query=query, top_k=5)

    # Build context_items for prompt_formatter
    context_items = [
        {
            "page_number": res.get("page_number", "?"),
            "sentence_chunk": res["text"]
        }
        for res in results
    ]

    # Build the full prompt
    prompt = prompt_formatter(query=query, context_items=context_items)

    # Move prompt to device
    device = "cuda" if torch.cuda.is_available() else "cpu"
    inputs = tokenizer(prompt, return_tensors="pt").to(device)

    # Generate output
    with torch.no_grad():
        outputs = llm_model.generate(
            **inputs,
            temperature=0.7,
            do_sample=True,
            max_new_tokens=512,
            pad_token_id=tokenizer.eos_token_id,
        )

    output_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Try to remove the prompt portion
    start_idx = output_text.find("[Slide 1]")
    if start_idx != -1:
        answer = output_text[start_idx:].strip()
    else:
        answer = output_text.replace(prompt, "").strip()

    contexts = [item["sentence_chunk"] for item in context_items]
    return answer, contexts


In [62]:
evaluation_data = []

print("Generating SARAL RAG answers for evaluation...\n")
for question, ground_truth in zip(eval_questions, ground_truth_answers):
    print(f"Processing: {question[:60]}...")
    rag_answer, contexts = generate_saral_answer(question)

    evaluation_data.append({
        'question': question,
        'answer': rag_answer,
        'contexts': contexts,
        'ground_truth': ground_truth
    })


Generating SARAL RAG answers for evaluation...

Processing: Derive the ELBO objective function step-by-step....
Processing: Summarize the key equation for the KL Divergence....
Processing: Explain how diffusion models relate to variational inference...
Processing: What is the role of the reparameterization trick in this pap...
Processing: State the loss function used for optimization....


In [47]:
eval_dataset = Dataset.from_pandas(pd.DataFrame(evaluation_data))

metrics = [
    context_precision,
    context_recall,
    answer_relevancy,
    faithfulness
]

if context_entity_recall is not None:
    metrics.append(context_entity_recall)
if noise_robustness is not None:
    metrics.append(noise_robustness)

print("\nRunning RAGAS evaluation...")
results = evaluate(
    dataset=eval_dataset,
    metrics=metrics,
)
results_df = results.to_pandas()



Running RAGAS evaluation...


Evaluating: 100%|██████████| 25/25 [00:29<00:00,  1.19s/it]


In [48]:
print("\n" + "="*80)
print(" SARAL RAG EVALUATION RESULTS")
print("="*80)

# Collect numeric columns
metric_cols = [c for c in results_df.columns if c not in ['user_input', 'retrieved_contexts', 'response', 'reference']]

# Clean result table
clean_results = pd.DataFrame()
clean_results['Question'] = [f"Q{i+1}: {q[:50]}..." for i, q in enumerate(results_df['user_input'])]
for col in metric_cols:
    clean_results[col.replace('_', ' ').title()] = results_df[col].round(3)

print("\n📋 INDIVIDUAL QUESTION PERFORMANCE")
print("-"*80)
print(clean_results.to_string(index=False))

# Overall averages
print("\n📊 OVERALL AVERAGE SCORES")
print("-"*50)
for col in metric_cols:
    avg = results_df[col].mean()
    print(f"{col.replace('_', ' ').title():<25}: {avg:.3f}")

# Save to CSV
detailed_results = pd.DataFrame(evaluation_data)
for col in metric_cols:
    detailed_results[col] = results_df[col].values
detailed_results.to_csv('saral_rag_evaluation.csv', index=False)

print("\n💾 Results saved to 'saral_rag_evaluation.csv'")



 SARAL RAG EVALUATION RESULTS

📋 INDIVIDUAL QUESTION PERFORMANCE
--------------------------------------------------------------------------------
                                                 Question  Context Precision  Context Recall  Answer Relevancy  Faithfulness  Context Entity Recall
  Q1: Derive the ELBO objective function step-by-step....              1.000             1.0             0.954         0.800                  0.200
 Q2: Summarize the key equation for the KL Divergence....              0.250             1.0             0.000         0.125                  0.250
Q3: Explain how diffusion models relate to variational...              1.000             1.0             0.956         0.778                  0.000
Q4: What is the role of the reparameterization trick i...              1.000             1.0             0.961         1.000                  0.167
    Q5: State the loss function used for optimization....              0.639             1.0             0.858   

In [49]:
best_metric = max(metric_cols, key=lambda c: results_df[c].mean())
worst_metric = min(metric_cols, key=lambda c: results_df[c].mean())

print(f"\n🚀 Best metric: {best_metric}")
print(f"⚠️ Weakest metric: {worst_metric}")



🚀 Best metric: context_recall
⚠️ Weakest metric: context_entity_recall


In [51]:
#!pip install gradio
import gradio as gr


ImportError: cannot import name 'ErrorWrapper' from 'fastapi._compat' (/media/data_dump/conda/miniconda3/envs/SARAL/lib/python3.10/site-packages/fastapi/_compat/__init__.py)

In [55]:
def saral_chat_interface(query, top_k=5, audience="general"):
    """
    Gradio interface function for SARAL chatbot.
    """
    try:
        answer, context_used = ask_saral_query(query=query, top_k=top_k, show_prompt=False)
        context_str = "\n\n".join([f"📄 Page {c['page_number']}: {c['sentence_chunk'][:300]}..." for c in context_used])
        return answer, context_str
    except Exception as e:
        return f"⚠️ Error: {str(e)}", ""


In [56]:
def process_pdf(file_obj):
    """
    Called when a new PDF is uploaded.
    Extracts text, chunks it, creates embeddings, and stores in Chroma.
    """
    import fitz
    import re
    from tqdm.auto import tqdm

    pdf_path = file_obj.name
    doc = fitz.open(pdf_path)

    pages_and_texts = []
    for page_number, page in tqdm(enumerate(doc)):
        text = page.get_text()
        text = text.replace("\n", " ").strip()
        pages_and_texts.append({
            "page_number": page_number,
            "text": text,
        })

    # Chunk + embed + store in Chroma (reusing your existing functions)
    # For simplicity, you can call your existing embedding code here
    # Example:
    # rebuild_chromadb_from_pdf(pages_and_texts)

    return f"✅ Uploaded and processed {len(pages_and_texts)} pages from: {pdf_path}"


In [57]:
with gr.Blocks(theme=gr.themes.Soft()) as saral_ui:
    gr.Markdown(
        """
        # 🧠 SARAL — Audience-Adaptive Research Communicator  
        Upload a paper or ask a question to generate scripts, slides, or summaries.
        ---
        """
    )

    with gr.Tab("Chat with SARAL"):
        with gr.Row():
            query_box = gr.Textbox(
                label="Enter your request",
                placeholder="e.g., Make a 5-slide talk for graduate students focusing on methods...",
                lines=3
            )

        with gr.Row():
            audience_box = gr.Dropdown(
                ["general", "technical", "policymaker", "press release", "student"],
                value="general",
                label="Target Audience"
            )
            topk_slider = gr.Slider(3, 10, value=5, step=1, label="Top-k Chunks Retrieved")

        with gr.Row():
            answer_box = gr.Textbox(label="SARAL Response", lines=20)
            context_box = gr.Textbox(label="Retrieved Context (Provenance)", lines=20)

        submit_btn = gr.Button("💬 Generate")

        submit_btn.click(
            fn=saral_chat_interface,
            inputs=[query_box, topk_slider, audience_box],
            outputs=[answer_box, context_box]
        )

    with gr.Tab("Upload Paper"):
        pdf_input = gr.File(label="Upload PDF Paper")
        upload_output = gr.Textbox(label="Upload Status")
        pdf_input.upload(process_pdf, inputs=pdf_input, outputs=upload_output)

    gr.Markdown("Built with ❤️ using Gemma + Chroma + Gradio")


In [58]:
saral_ui.launch(share=True, debug=True)


* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://70824d6061c571da85.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "/media/data_dump/conda/miniconda3/envs/SARAL/lib/python3.10/site-packages/uvicorn/protocols/http/httptools_impl.py", line 409, in run_asgi
    result = await app(  # type: ignore[func-returns-value]
  File "/media/data_dump/conda/miniconda3/envs/SARAL/lib/python3.10/site-packages/uvicorn/middleware/proxy_headers.py", line 60, in __call__
    return await self.app(scope, receive, send)
  File "/media/data_dump/conda/miniconda3/envs/SARAL/lib/python3.10/site-packages/fastapi/applications.py", line 1134, in __call__
    response_model_exclude_defaults: bool = False,
  File "/media/data_dump/conda/miniconda3/envs/SARAL/lib/python3.10/site-packages/starlette/applications.py", line 113, in __call__
    await self.middleware_stack(scope, receive, send)
  File "/media/data_dump/conda/miniconda3/envs/SARAL/lib/python3.10/site-packages/starlette/middleware/errors.py", line 186, in __call__
    # to optionally rais

Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://70824d6061c571da85.gradio.live
